In [1]:
import os
from openai import OpenAI

client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key="sk-db3f839bc51e459dae3aab49d1a779e2",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# 通过 messages 数组实现上下文管理
messages = [
    {'role': 'user', 'content': '你好'}
]

completion = client.chat.completions.create(
    model="deepseek-r1",  # 此处以 deepseek-r1 为例，可按需更换模型名称。
    messages=messages
)

print("="*20+"第一轮对话"+"="*20)
# 通过reasoning_content字段打印思考过程
print("="*20+"思考过程"+"="*20)
print(completion.choices[0].message.reasoning_content)
# 通过content字段打印最终答案
print("="*20+"最终答案"+"="*20)
print(completion.choices[0].message.content)

messages.append({'role': 'assistant', 'content': completion.choices[0].message.content})
messages.append({'role': 'user', 'content': '你是谁'})
print("="*20+"第二轮对话"+"="*20)
completion = client.chat.completions.create(
    model="deepseek-r1",  # 此处以 deepseek-r1 为例，可按需更换模型名称。
    messages=messages
)
# 通过reasoning_content字段打印思考过程
print("="*20+"思考过程"+"="*20)
print(completion.choices[0].message.reasoning_content)
# 通过content字段打印最终答案
print("="*20+"最终答案"+"="*20)
print(completion.choices[0].message.content)

====================第一轮对话====================
====================思考过程====================
你好！很高兴见到你，有什么我可以帮忙的吗？无论是学习、工作还是生活中的问题，都可以告诉我哦！😊
====================最终答案====================
你好！很高兴见到你，有什么我可以帮忙的吗？无论是学习、工作还是生活中的问题，都可以告诉我哦！😊
====================第二轮对话====================
====================思考过程====================
好的，用户现在问“你是谁”，看起来他们想更了解我的身份和功能。之前用户先用中文打招呼，我也用中文回复了，然后他们继续用中文提问，所以整个对话都是中文的。

首先，我需要明确用户的需求。用户可能已经知道我是AI助手，但想了解更详细的信息，比如我的开发团队、功能、数据隐私政策等。也有可能他们是在测试我的回答能力，或者对AI技术本身感兴趣。

接下来，我要考虑之前的对话历史。用户第一次打招呼“你好”，我回应了，然后他们直接问“你是谁”。这说明用户可能在初次接触后，想确认我的身份和可信度。这时候需要提供一个全面但简洁的回答，涵盖我的基本信息和能力，同时让用户感到放心。

然后，我需要确保回答准确无误，符合公司的指导方针。要提到我是由中国的深度求索（DeepSeek）公司开发的智能助手，强调多领域的帮助能力，比如回答问题、提供建议、学习辅助等。同时，必须提及数据隐私和安全性，这是用户可能关心的点。

另外，用户可能希望知道我的局限性和更新时间，这样他们可以合理预期我的回答范围。比如，我的知识截止到2023年12月，无法访问实时信息，这些都需要明确说明，避免误解。

还要保持友好和开放的态度，鼓励用户继续提问，让他们感到被欢迎和支持。使用表情符号可以增加亲切感，但根据用户的风格调整，如果之前用户用了表情，可以适当回应。

最后，检查回答是否符合中文表达习惯，避免技术术语过多，确保清晰易懂。同时，结构要条理分明，分点说明，方便用户快速获取信息。

总结来说，回答需要涵盖身份、功能、隐私、局限性，以及鼓励互动，所有内容都要准确、友好且易于理解。
=

In [16]:
import re

def extract_character_features(text):
    prompt = f"""
请阅读以下中文小说文本，并识别所有明确提到的“角色名”及其“特征描述”，并将结果用 JSON 结构返回。
JSON的键值对为"角色名X": "xxx", "特征": "xxx"
例子：
  {{"角色名1": "瑞奇",
  "特征": "25岁黑色短发的男性",
  
  "角色名2": "",
  "特征2": "",
  
  "角色名3": "",
  "特征3": ""}}
注意：
1. 角色名应当真实在文本中出现。
2. 特征包括年龄、外貌、衣着、行为、情绪等明显描述。
3. 不需要编造额外信息。

小说内容如下：
{text}
"""
    client = OpenAI(
        # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
        api_key="sk-db3f839bc51e459dae3aab49d1a779e2",
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )
    
    response = client.chat.completions.create(
        model="deepseek-r1",  # 或 "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "你是一个擅长文本分析的中文助手。"},
            {"role": "user", "content": prompt}
        ],
        response_format={"type": "json_object"},
        temperature=0.5
    )

    result = response.choices[0].message.content
    
    # 尝试解析 JSON
    try:
        # 步骤 1：去除 Markdown 包裹符
        cleaned = re.sub(r'^```json|```$', '', result.strip(), flags=re.MULTILINE).strip()
        
        # 步骤 2：解析为 Python 对象
        json_data = json.loads(cleaned)
        with open("角色信息.json", "w", encoding="utf-8") as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False)
        return json_data
    except json.JSONDecodeError:
        print("返回内容不是合法的 JSON，可以考虑用正则或人工检查")
        return result


text = "瑞奇是一个25岁的青年，黑色短发，穿着绿色的夹克。他面无表情地走在乡间小路上。艾米躲在一棵大树后面，眼神中透着不安。"
extract_character_features(text)

NameError: name 're' is not defined

In [42]:
def update_config_with_characters(character_data, config_path="..\config.json"):
    # 加载已有 config.json
    with open(config_path, "r", encoding="utf-8") as f:
        config = json.load(f)

    # 清空原有的角色字段（角色名1~10 和 特征1~10）
    for i in range(1, 11):
        config[f"角色名{i}"] = ""
        config[f"特征{i}"] = ""

    # 写入新的角色信息
    for idx, (role_key, feature_value) in enumerate(character_data.items()):
        if "角色名" in role_key:
            role_index = idx // 2 + 1
            config[f"角色名{role_index}"] = feature_value
        elif "特征" in role_key:
            feature_index = idx // 2 + 1
            config[f"特征{feature_index}"] = feature_value

    # 保存回 config.json
    with open(config_path, "w", encoding="utf-8") as f:
        json.dump(config, f, indent=2, ensure_ascii=False)
    print("✅ config.json 已成功更新。")


from openai import OpenAI
import re
import os
import json


def extract_character_features(text):
    prompt = f"""
请阅读以下中文小说文本，并识别所有明确提到的“角色名”及其“特征描述”，并将结果用 JSON 结构返回。
JSON的键值对为"角色名": "xxx", "特征": "xxx"
注意：
1. 角色名应当真实在文本中出现。
2. 特征仅包括角色的数量、年龄、外貌、衣着，切记不要包含任何人物关系的描述
3. 如果人物的特征不足甚至没有的话，可以根据文本内容编造。
例子：
  {{"角色名1": "帕奇",
  "特征1": "一个20岁的黑发男子",
  
  "角色名2": "",
  "特征2": "",
  
  "角色名3": "",
  "特征3": ""}}

小说内容如下：
{text}
"""
    client = OpenAI(
        # 如果没有配置环境变量，请用百炼API Key替换：api_key="sk-xxx"
        api_key="sk-db3f839bc51e459dae3aab49d1a779e2",
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
    )

    response = client.chat.completions.create(
        model="qwen-plus",  # 或 "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "你是一个擅长文本分析的中文助手。"},
            {"role": "user", "content": prompt}
        ],
        response_format={"type": "json_object"},
        temperature=0.5
    )

    result = response.choices[0].message.content

    # 尝试解析 JSON
    try:
        # 步骤 1：去除 Markdown 包裹符
        cleaned = re.sub(r'^```json|```$', '', result.strip(), flags=re.MULTILINE).strip()
        # 步骤 2：解析为 Python 对象
        json_data = json.loads(cleaned)
        with open("角色信息.json", "w", encoding="utf-8") as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False)
        return json_data
    except json.JSONDecodeError:
        print("返回内容不是合法的 JSON，可以考虑用正则或人工检查")
        return result


with open("../input.txt", "r", encoding="utf-8") as file:
    text = file.read()
    character_info = extract_character_features(text)
    print(character_info)
    update_config_with_characters(character_info)

{'角色名1': '国王', '特征1': '一个渴望孩子的中年男子，衣着华丽', '角色名2': '王后', '特征2': '一位美丽、善良的女子，后来因病去世', '角色名3': '白雪公主', '特征3': '皮肤白皙如雪，双颊红润如苹果，头发乌黑柔顺的美少女，约十几岁，穿着普通的衣物', '角色名4': '新王后', '特征4': '外表美丽但性格骄傲暴躁的女巫，精通法术，穿着华丽', '角色名5': '七个小矮人', '特征5': '七个身材矮小的男人，具体年龄不详，穿着简单的工作服', '角色名6': '邻国王子', '特征6': '英俊潇洒的年轻男子，约二十多岁，穿着高贵的王子服饰', '角色名7': '魔镜', '特征7': '一面能回答问题的奇特镜子，无具体外貌或年龄'}
✅ config.json 已成功更新。


In [47]:
from docx import Document

def read_docx(file_path):
    return [
        paragraph.text
        for paragraph in Document(file_path).paragraphs
        if paragraph.text.strip()
    ]
paragraphs = read_docx("../input.docx")

print(paragraphs)

sentences = []
for paragraph in paragraphs:
    sentences.extend([sent.text for sent in nlp(paragraph).sents])


['在遥远的一个国度里，住着一个国王和王后，他们渴望有一个孩子。于是很诚意的向上苍祈祷。', '“上帝啊！我们都是好国王好王后，请您赐给我们一个孩子吧！”不久以后，王后果然生下了一个可爱的小公主，这个女孩的皮肤白得像雪一般，双颊红得有如苹果，头发乌黑柔顺，因此，国王和王后就把她取名为“白雪公主”。', '全国的人民都为白雪公主深深祝福。', '白雪公主在国王白雪公主和王后的宠爱之下，逐渐长大了，终于成了一个人见人爱的美少女。白雪公主非常善良、有爱心、她经常和动物一起玩耍。森林的动物，像小鹿、小兔子、松鼠、小鸟都喜欢白雪公主，因为白雪公主会给它们吃食物，还会讲故事给它们听。个性善良犹如天使般的白雪公主，过着幸福快乐的生活。', '可是，好景不长，白雪公主的母亲生病去世了。', '国王为了白雪公主就迎娶了一位新王后，可是，这位新王后却是个精通法术的女巫。她虽然很美丽，但是个性很骄傲、暴躁。尤其她最恨别人比她美丽。', '“这是你新母后！”', '当国王向白雪公主介绍新王后时，她还正为死去的母后感到悲伤呢。新王后有一面很奇特的镜子，从镜子里可以得到一切你想知道的答案。', '所以，王后经常对着镜子问：“魔镜、魔镜，谁是世界上最美丽的女人？”', '“全世界最美的女人就是你，王后。”', '可是，有一天，当王后再问魔镜同样的问题时，魔镜却回答说：', '“现在白雪公主比你美丽。”', '新王后听了非常生气。', '“可恶，怎么白雪公主和七个小矮人可以有人比我更美丽，我一定要把她除去。”', '于是，她就命令宫廷的武士说：', '“我不想再看到白雪公主了，你找个借口，把她带到森林里偷偷杀掉。杀了以后，把她的心和舌头带回来，做为你杀死她的证据。听到没有？不可以有差错……”', '“是的，王后……”', '武士听了这话之后，就真的把白雪公主带到森林里去了。', '当武士抽出刀来杀公主的时候，他看到正在采花的公主，纯洁，善良，犹如天使一般，武士不忍心杀她，就向白雪公主说：', '“皇后命令我杀掉公主你，可是我实在狠不下心，所以你还是往森林里逃走吧！”', '说完武士见到一头猪，就跑过去宰了它，并取下心和舌头以作为证据。', '之后，武士便回皇宫去了。', '听到猫头鹰叫声的白雪公主，越走越觉得森林好可怕。', '突然，眼前有一栋小木屋，于是便又惊又喜的叫着：', '“啊，是小木屋！